In [ ]:
using Revise, LazySets, Polyhedra, CDDLib

----

In [ ]:
n = 4
X = convert(HPolytope, rand(Hyperrectangle, dim=n))
s = [project(X, [i, i+1]) for i in 1:2:Int(floor(n/2))]
typeof(s)

In [ ]:
push!(s, project(X, [5]))

In [ ]:
using LazySets.Approximations: AbstractDirections
using LazySets: dim

function _overapproximate_zonotope_cpa(X::LazySet, ::Type{<:Zonotope}, dir::AbstractDirections)
    n = LazySets.dim(X)
    nblocks = Int(floor(n/2))
    πX_2D = [project(X, [i, i+1]) for i in 1:2:nblocks]

    # overapproximate 2D blocks
    Z_2D = [_overapproximate_zonotope_vrep(poly, Zonotope, dir) for poly in πX_2D]

    if iseven(n)
        return CartesianProductArray(Z_2D)
    else
        # odd case projects onto an interval
        z = project(X, [n])
        πX_n = _overapproximate_zonotope_vrep(z, Zonotope, BoxDirections(1))
        πX_n = overapprximate(Interval, πX_n)
        return CartesianProductArray(vcat(Z_2D, πX_n))
    end
end

---

## TESTS

In [4]:
using Revise, LazySets, Polyhedra, CDDLib
using LazySets: Interval

In [10]:
B = rand(Interval, dim=1)
dirs = BoxDirections(1);
Y1 = overapproximate(B, Zonotope, dirs)
Y2 = overapproximate(B, Zonotope, dirs, algorithm="vrep")
Y3 = overapproximate(B, Zonotope, dirs, algorithm="cpa");

In [13]:
Y3

CartesianProductArray{Float64,AbstractZonotope{Float64}}(AbstractZonotope{Float64}[Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.987861, -0.797246])])

In [12]:
Y2

Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([-0.8925533265516776], [0.09530701636074967])

In [ ]:
# ----------
B = rand(Ball1, dim=1)
dirs = BoxDirections(1);
overapproximate(B, Zonotope, dirs)
overapproximate(B, Zonotope, dirs, algorithm="vrep")
overapproximate(B, Zonotope, dirs, algorithm="cpa")

In [ ]:
B = rand(Ball1, dim=1)
dirs = BoxDirections(1);
overapproximate(B, Zonotope, dirs)
overapproximate(B, Zonotope, dirs, algorithm="vrep")
overapproximate(B, Zonotope, dirs, algorithm="cpa")

# ----------



In [ ]:
B = rand(Ball1, dim=2)
dirs = BoxDirections(2);

# should use vrep method
overapproximate(B, Zonotope, dirs)

# should use vrep method
overapproximate(B, Zonotope, dirs, algorithm="vrep")

# should use cpa method
_overapproximate_zonotope_cpa(B, Zonotope, dirs, algorithm="cpa")

In [ ]:
B = rand(Ball1, dim=3)
dirs = BoxDirections(3);

# should use vrep method
overapproximate(B, Zonotope, dirs)

# should use vrep method
overapproximate(B, Zonotope, dirs, algorithm="vrep")

# should use cpa method
_overapproximate_zonotope_cpa(B, Zonotope, dirs, algorithm="cpa")

In [15]:
convert(Zonotope, rand(Interval))

Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([0.1603181947923759], [0.2224425240669668])

Supongamos que $n = 3$

In [32]:
using LazySets: dim

X = rand(Zonotope, dim=3)
Z = overapproximate(X, Zonotope, OctDirections, algorithm="cpa");

In [35]:
using Plots

In [38]:
Z12 = Z.array[1]
Z3 = Z.array[2]

Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([1.4167303361303372], [2.9671344944217495])

In [39]:
new_center = vcat(Z12.center, Z3.center)
new_generators = vcat(Z12.center, Z3.center)

3-element Array{Float64,1}:
 1.905973395157175
 0.23850446172771922
 1.4167303361303372

In [40]:
Z12.generators

2×4 Array{Float64,2}:
 1.31872   0.0391875  3.31968  0.0
 1.31872  -0.0391875  0.0      2.17705

In [41]:
Z3.generators

1×1 Array{Float64,2}:
 2.9671344944217495

In [42]:
using LinearAlgebra

In [47]:
Diagonal([Z12.generators, Z3.generators])

2×2 Diagonal{Array{Float64,2},Array{Array{Float64,2},1}}:
 [1.31872 0.0391875 3.31968 0.0; 1.31872 -0.0391875 0.0 2.17705]      ⋅     
         ⋅                                                        [2.96713]

In [49]:
using SparseArrays

In [55]:
M = blockdiag(sparse(Z12.generators), sparse(Z3.generators));
Matrix(M)

3×5 Array{Float64,2}:
 1.31872   0.0391875  3.31968  0.0      0.0
 1.31872  -0.0391875  0.0      2.17705  0.0
 0.0       0.0        0.0      0.0      2.96713

----

In [60]:
using SparseArrays

In [76]:
function _convert(::Type{Zonotope}, cpa::CartesianProductArray{N,
                 <:AbstractZonotope{N}}) where {N}
    array = cpa.array
    c = sparse(reduce(vcat, LazySets.center.(array)))
    G = reduce(blockdiag, sparse.(genmat.(array)))
    return Zonotope(c, G)
end

_convert (generic function with 1 method)

In [79]:
x = sparsevec([1, 2], [4.0, 5.0], 3)
vcat(x, x)

6-element SparseVector{Float64,Int64} with 4 stored entries:
  [1]  =  4.0
  [2]  =  5.0
  [4]  =  4.0
  [5]  =  5.0

In [77]:
x = [rand(Zonotope) for _ in 1:5];
_convert(Zonotope, CartesianProductArray(x))

Zonotope{Float64,SparseVector{Float64,Int64},SparseMatrixCSC{Float64,Int64}}(  [1 ]  =  -2.45236
  [2 ]  =  0.629654
  [3 ]  =  0.827285
  [4 ]  =  -1.09923
  [5 ]  =  2.20895
  [6 ]  =  0.418269
  [7 ]  =  -0.142181
  [8 ]  =  0.104313
  [9 ]  =  -1.34329
  [10]  =  -1.37916, 
  [1 ,  1]  =  1.12583
  [2 ,  1]  =  1.96283
  [1 ,  2]  =  -0.0812319
  [2 ,  2]  =  0.167425
  [1 ,  3]  =  1.25032
  [2 ,  3]  =  -0.698021
  [1 ,  4]  =  0.0191633
  [2 ,  4]  =  -0.629271
  [3 ,  5]  =  0.0405677
  [4 ,  5]  =  1.27751
  [3 ,  6]  =  -0.0223759
  [4 ,  6]  =  0.0646973
  ⋮
  [8 ,  9]  =  -0.568358
  [7 , 10]  =  -0.590919
  [8 , 10]  =  -0.487233
  [7 , 11]  =  0.142221
  [8 , 11]  =  -2.59975
  [9 , 12]  =  1.092
  [10, 12]  =  0.451703
  [9 , 13]  =  -1.5769
  [10, 13]  =  -0.80627
  [9 , 14]  =  0.0238759
  [10, 14]  =  0.261564
  [9 , 15]  =  -1.07648
  [10, 15]  =  -0.709592)

In [70]:
reduce(blockdiag, sparse.(genmat.(x)))

10×16 SparseMatrixCSC{Float64,Int64} with 32 stored entries:
  [1 ,  1]  =  -2.00993
  [2 ,  1]  =  0.937552
  [1 ,  2]  =  -1.00395
  [2 ,  2]  =  3.25927
  [3 ,  3]  =  0.0844815
  [4 ,  3]  =  -0.804111
  [3 ,  4]  =  -0.470436
  [4 ,  4]  =  -0.656306
  [3 ,  5]  =  0.999216
  [4 ,  5]  =  0.105169
  [3 ,  6]  =  0.768036
  [4 ,  6]  =  0.394399
  ⋮
  [8 , 10]  =  1.10435
  [7 , 11]  =  0.914492
  [8 , 11]  =  -0.971978
  [7 , 12]  =  2.26006
  [8 , 12]  =  -1.0348
  [9 , 13]  =  -0.217073
  [10, 13]  =  -0.751995
  [9 , 14]  =  -0.0766718
  [10, 14]  =  -0.658735
  [9 , 15]  =  -0.656569
  [10, 15]  =  0.73828
  [9 , 16]  =  -0.0174222
  [10, 16]  =  0.0105527